In [1]:
from transformers import MBartForConditionalGeneration, MBartTokenizer, AutoTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BertTokenizer, BartForConditionalGeneration

import torch
from torch.utils.data import random_split
import json
from datasets import load_metric
import numpy as np
# metric = load_metric("bleu")
import random


In [2]:
# data_pre_fix = '/work/u9296553/aics/seq2seq/data_tailo_number'
# data_pre_fix = '/work/u9296553/aics/seq2seq/data'
data_pre_fix = '/work/u9296553/aics/seq2seq/data_tgb'

fp = open(f'{data_pre_fix}/test_data.json')
eval_dataset = json.load(fp)
print(len(eval_dataset))
eval_dataset[2]

490


{'tai_lo': 'beh4 hoo7 hng7 loo7 lai5 e5 iu5 kheh4 kam2 siu7 koo2 khenn1 tsai7 te7 khip4 in2 lat8',
 'tai_wen': '要讓遠道來的遊客感受古坑在地魅力'}

In [3]:
src_lang="en_XX"
tgt_lang="zh_CN"

In [62]:
# model_name = '/work/u9296553/aics/seq2seq/tailo_to_taiwen_100/checkpoint-22500'
# model_name  = '/work/u9296553/aics/seq2seq/tailo_to_taiwen_test/checkpoint-17000'
# tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang="tw_TL", tgt_lang="tw_TW")

In [4]:
# model_name = '/work/u9296553/aics/seq2seq/tailo_to_taiwen_origin_tokenizer_with_unk_resize/checkpoint-1448'
# tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang=src_lang, tgt_lang=tgt_lang)

In [5]:
model_name = '/work/u9296553/aics/seq2seq/checkpoints/TGB_tailo_to_zh/checkpoint-12033'
tokenizer = BertTokenizer.from_pretrained(model_name, use_fast=True)

In [5]:
# def data_collator(features:list):

#     labels = [f["tai_wen"] for f in features]
#     inputs = [f["tai_lo"] for f in features]

#     batch = tokenizer.prepare_seq2seq_batch(src_texts=inputs, src_lang="tw_TL", tgt_lang="tw_TW", tgt_texts=labels, max_length=32, max_target_length=32)

#     for k in batch:
#         batch[k] = torch.tensor(batch[k])

#     return batch

# data_collator(eval_dataset)['input_ids'][0]

In [6]:
device = "cuda"
# model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

In [7]:
predictions  = []
groundtruth = []

batch_size = 64
max_length = 48
for i in range(0, len(eval_dataset), batch_size):
    # input_texts = [eval_dataset[k]['tai_lo'] for k in range(i, i + batch_size) if k < len(eval_dataset)]
    # labels = [eval_dataset[k]['tai_wen'] for k in range(i, i + batch_size) if k < len(eval_dataset)]

    input_texts = [eval_dataset[k]['tai_wen'] for k in range(i, i + batch_size) if k < len(eval_dataset)]
    labels = [eval_dataset[k]['tai_lo'] for k in range(i, i + batch_size) if k < len(eval_dataset)]

    

    groundtruth.extend(labels)
    encoded_input = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    if 'token_type_ids' in encoded_input : del encoded_input['token_type_ids']
    encoded_input = encoded_input.to(device)
    generated_tokens = model.generate(**encoded_input, early_stopping=True, max_length=max_length)

    # generated_tokens = model.generate(**encoded_input, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang], early_stopping=True, max_length=48)
    # generated_tokens = model.generate(**encoded_input, forced_bos_token_id=21210, early_stopping=True, max_length=48)

    generated_tokens = [i[i != tokenizer.cls_token_id ] for i in generated_tokens]
    generated_tokens = [i[i != tokenizer.sep_token_id ] for i in generated_tokens]
    generated_tokens = [i[i != tokenizer.pad_token_id ] for i in generated_tokens]
    generated_tokens = [i[i != tokenizer.unk_token_id ] for i in generated_tokens]

    output_texts = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)


    output_texts = [''.join(output_text) for output_text in output_texts]
    # output_text = ''.join(output_text)
    # print(output_text)
    predictions.extend(output_texts)

    print(f'\r {i}', end='')

print()
# results = metric.compute(predictions=predictions, references=references)
# print(results["bleu"])

 448


In [8]:
print(groundtruth[:10])
print(predictions[:10])

['in1 ui7 a1 tiong1 kap4 a1 kok4 kau1 tai3 ai3 lau7 jiat8', 'bin5 tsiong3 khi3 kip4 pai7 hai7', 'beh4 hoo7 hng7 loo7 lai5 e5 iu5 kheh4 kam2 siu7 koo2 khenn1 tsai7 te7 khip4 in2 lat8', 'ki5 sit8 tsit4 ui7 khai1 long2 e5 siau3 lu2', 'siunn7 be7 kau3 tshu3 tsu2 khuann3 tioh8 tshim1 siu7 kam2 tong7', 'tsun7 tioh8 kong1 si7 e5 lik8 su2 kik8 tann2 piann3 tai5 uan5 jin5 bin5 e5 lik8 su2', 'bok8 tsing5 long2 si7 su2 iong7 te7 ha7 tsui2 im1 tshan5', 'sam1 lip8 sin1 bun5 tai5', 'ka1 siang7 hong1 thai1 sai1 lam5 khi3 liu5 e5 ing2 hiang2', 'iau2 u7 lang5 pang3 phau3 a2 khong3 gi7']
['in1 ui7 a1 tiong1 kap4 a1 kok4 kau1 tai3 ai3 lau7 jiat8', 'bin5 tsiong3 khi3 kip4 pai7 hai7', 'beh4 ho7 uan2 to7 lai5 e5 iu5 kheh4 kam2 siu7 koo2 khenn1 tsai7 te7 khip4 in2 lat8', 'ki5 sit8 tsit4 ui7 khai1 kiann7 e5 siau3 lu2', 'siunn7 be7 kau3 tshu3 tsu2 khuann3 tioh8 tshim1 siu7 kam2 tong7', 'khuann3 tioh8 kong1 si7 e5 lik8 su2 tua7 kiok8 teh4 phah4 piann3 tai5 uan5 jin5 bin5 e5 lik4 su2', 'bok8 tsing5 long2 si7 su2

In [9]:
predictions = [p.replace(' ','') for p in predictions]
print(len(predictions))

print(groundtruth[:10])
print(predictions[:10])

490
['因為阿中和阿國交代要熱鬧', '民眾氣急敗壞', '要讓遠道來的遊客感受古坑在地魅力', '其實這位開朗的少女', '想不到屋主看到深受感動', '看到公視的歷史大劇打拚台灣人民的歷史', '目前都是使用地下水灌溉', '三立新聞台', '加上颱風西南氣流的影響', '還有人放鞭炮抗議']
['因為阿中和阿國交代愛熱鬧', '民眾氣急敗壞', '要讓遠路來的遊客感受古坑在地吸引力', '其實這位花都的少女', '想不到屋主看到深受感動', '隨著公視的歷史劇打拼台灣人民的血史', '目前都是使用地下水淹田', '三立新聞台', '加上颱風西南氣流的影響', '還有人放鞭炮抗議']


In [13]:
fp = open('./TGB_data.txt', 'w')

In [14]:
for i in range(len(predictions)):
    # print('------台羅數字調-------')
    print(eval_dataset[i]['tai_lo'], file=fp)
    # print('-----Pred中文---------')
    print(predictions[i], file=fp)
    print(groundtruth[i], file=fp)
    print('-'*10, file=fp)
    # input()
fp.close()

In [70]:
normal_groundtruth = []

replace_map = {
    'á' : 'a',
    'à' : 'a',
    'â' : 'a',
    'ǎ' : 'a',
    'a̋' : 'a',
    'ā' : 'a',
    'a̍h' : 'ah',
    'a̍' : 'a',

    'é' : 'e',
    'è' : 'e',
    'ê' : 'e',
    'ě' : 'e',
    'ē' : 'e',
    'e̋' : 'e',
    'e̍h' : 'eh',
    'e̍' : 'e',


    'í' : 'i',
    'ì' : 'i',
    'î' : 'i',
    'ǐ' : 'i',
    'ī' : 'i',
    'i̋' : 'i',
    'i̍h' : 'ih',
    'i̍' : 'i',


    'ó' : 'o',
    'ò' : 'o',
    'ô' : 'o',
    'ǒ' : 'o',
    'ō' : 'o',
    'ő' : 'o',
    'o̍h' : 'oh',
    'o̍' : 'o',


    'ú' : 'u',
    'ù' : 'u',
    'û' : 'u',
    'ǔ' : 'u',
    'ū' : 'u',
    'ű' : 'u',
    'u̍h' : 'uh',
    'u̍' : 'u',


    'ḿ' : 'm',
    'm̀' : 'm',
    'm̂' : 'm',
    'm̌' : 'm',
    'm̄' : 'm',
    'm̋' : 'm',
    'm̍h' : 'mh',
    'm̍' : 'm',


    'ń' : 'n',
    'ǹ' : 'n',
    'n̂' : 'n',
    'ň' : 'n',
    'n̄' : 'n',
    'n̋' : 'n',
    'n̍h' : 'nh',
    'n̍' : 'n',

}

for i in range(len(groundtruth)):
    
    normalize_gt = groundtruth[i]

    for k, v in replace_map.items():
        normalize_gt = normalize_gt.replace(k, v)

    normal_groundtruth.append(normalize_gt)


In [71]:
print(groundtruth[:10])
print(predictions[:10])

['因為阿中和阿國交代要熱鬧', '民眾氣急敗壞', '要讓遠道來的遊客感受古坑在地魅力', '其實這位開朗的少女', '想不到屋主看到深受感動', '看到公視的歷史大劇打拚台灣人民的歷史', '目前都是使用地下水灌溉', '三立新聞台', '加上颱風西南氣流的影響', '還有人放鞭炮抗議']
['因為阿中和阿國交代要熱鬧', '民眾氣級敗壞', '要讓遠路來的遊客感受古坑在地魅力', '其實這位開都的少女', '想不到屋主看到深受感動', '當著公視的歷史劇打拼台灣人民的歷歷史', '目前都是使用地下水淹田', '三立新聞台', '加上颱風西南氣流的影響', '還有人放鞭炮抗議']


In [72]:
import editdistance as ed
import csv
def cer_cal(groundtruth, hypothesis):
    err = 0
    tot = 0
    for p, t in zip(hypothesis, groundtruth):
        err += float(ed.eval(p.lower(), t.lower()))
        tot += len(t)
    return err / tot

cer = cer_cal(normal_groundtruth, predictions)
print(cer)

0.17186392629340894


In [107]:
for i in range(len(eval_dataset)):
    pred = predictions[i]
    pred = ''.join(pred)
    ref = references[i][0]
    ref = ''.join(ref)

    tai_lo = eval_dataset[i]['tai_lo']

    print(tai_lo)
    print(ref)
    print(pred)
    input()

tsǹg ji̍p piah kha kah mn̂g phāng
鑽入壁跤佮門縫
門跤佮門縫
tō kám kak khah bē hiah kuânn
就感覺較袂遐寒
就感覺 就感覺 就感覺
thiann tio̍h ba̍k kia̍h siann tō tsai iánn ū lâng king kuè
聽著木屐聲就知影有人經過
聽著木屐聲就知影有人經過
phóo phóo á lah
普普仔啦
普普仔啦
tsia̍h àm pá sé kha āu tshīng ba̍k kia̍h
食暗飽洗跤後穿木屐
後穿木屐
mā ē tàng óo tōo ún á khì tiò hî
嘛會當挖杜蚓仔去釣魚
去釣魚挖杜的聲挖杜
lāi bīn ê kang lâng suah tio̍h khì pa̍t king tshuē lîm sî kang lâi póo thiap bô khì ê ka pan
內面的工人煞著去別間揣臨時工來補貼無去的加班
我真去別間揣臨時工來補貼無去的加班
siōng kài kòo lâng uàn ê tō sī tsi̍t lē tsi̍t hiu
上蓋顧人怨的就是一例一休
上蓋顧人怨的就是一例一休
lîm siù hông koh kóng
林秀芃閣講
林秀芃閣講
tsóng kóng tsi̍t kù
總講一句
總講一句
lâi pin peh pah káu tsa̍p tshit hō tshiánn lâi jī tsa̍p hō kuī tâi pān lí
來賓八百九十七號請來二十號櫃台辦理
來賓八百九十七號請來二十號櫃台辦理
tán kàu guá tha̍k kok sió liáu āu
等到我讀國小了後
讀國小了後
koh beh iōng khàm thôo ê hong sik
閣欲用崁塗的方式
塗塗用崁塗的方式
khiā tī tiûnn guā thè guá ka iû
徛佇場外替我加油
徛佇場外替我加油
guá khuànn tsi̍t nî lâi lô thuân ê sîn kî ián tshut
我看一年來勞團的神奇演出
我看一年來勞團的神奇演出
tsit tsūn ài koh puē ìn nî uē
這陣愛閣背印尼話
背背行為
tāi pōo hūn tshù lāi tsa

KeyboardInterrupt: Interrupted by user

In [5]:

def predict(data, GPT_FIX=False):
    features = processor(data["speech"], sampling_rate=data["sampling_rate"], padding=True, return_tensors="pt")
    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)
    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits
    
    decoded_results = []
    for logit in logits:
        pred_ids = torch.argmax(logit, dim=-1)
        mask = pred_ids.ge(1).unsqueeze(-1).expand(logit.size())
        vocab_size = logit.size()[-1]
        voice_prob = torch.nn.functional.softmax((torch.masked_select(logit, mask).view(-1,vocab_size)),dim=-1)
        if GPT_FIX:
            gpt_input = torch.cat((torch.tensor([tokenizer.cls_token_id]).to(device),pred_ids[pred_ids>0]), 0)
            gpt_prob = torch.nn.functional.softmax(lm_model(gpt_input).logits, dim=-1)[:voice_prob.size()[0],:]
            comb_pred_ids = torch.argmax(gpt_prob*voice_prob, dim=-1)
        else: 
            comb_pred_ids = torch.argmax(voice_prob, dim=-1)
        # for wer
        decoded_results.append(processor.decode(comb_pred_ids, skip_special_tokens=True, spaces_between_special_tokens=True))
        # for cer
        # decoded_results.append(processor.decode(comb_pred_ids, skip_special_tokens=True))

    return decoded_results

In [8]:
import editdistance as ed
import csv


def eval_on_csv(csv_path):
    file = open(csv_path)
    csvreader = csv.reader(file)
    for i, row in enumerate(csvreader):
        if i == 0:
            continue
        wav_path = row[0]
        label = str(row[1])
        groundtruth.append(label)
        
        vdata = load_file_to_data(wav_path)
        pred = predict(vdata, GPT_FIX=False)
        
        
        pred = ''.join(pred)
        pred = pred.replace('[UNK]', '@')

        hypothesis.append(pred)

        # wer = wer_cal(groundtruth, hypothesis)
        # print(hypothesis)
        # print(groundtruth)
        # print(wer)
        # input()

        print(f'\r {i}', end='')

    wer = wer_cal(groundtruth, hypothesis)
    cer = cer_cal(groundtruth, hypothesis)
    print()
    print(wer)
    print(cer)


In [9]:
# # eval
# # eval_csv_path = '../data/vol1_vol2_ios_test.csv'
eval_csv_path = '/work/u9296553/aics/data/vol1_vol2_condenser_test_tai_lo.csv'


groundtruth = []
hypothesis = []


eval_on_csv(eval_csv_path)


print(len(groundtruth), len(hypothesis))






 5837
0.21773487051897536
0.09635377266734389
5837 5837


In [14]:
print(len(groundtruth), len(hypothesis))


5837 5837


In [27]:
normal_groundtruth = []
normal_hypothesis = []

replace_map = {
    'á' : 'a',
    'à' : 'a',
    'â' : 'a',
    'ǎ' : 'a',
    'a̋' : 'a',
    'ā' : 'a',
    'a̍h' : 'ah',
    'a̍' : 'a',

    'é' : 'e',
    'è' : 'e',
    'ê' : 'e',
    'ě' : 'e',
    'ē' : 'e',
    'e̋' : 'e',
    'e̍h' : 'eh',
    'e̍' : 'e',


    'í' : 'i',
    'ì' : 'i',
    'î' : 'i',
    'ǐ' : 'i',
    'ī' : 'i',
    'i̋' : 'i',
    'i̍h' : 'ih',
    'i̍' : 'i',


    'ó' : 'o',
    'ò' : 'o',
    'ô' : 'o',
    'ǒ' : 'o',
    'ō' : 'o',
    'ő' : 'o',
    'o̍h' : 'oh',
    'o̍' : 'o',


    'ú' : 'u',
    'ù' : 'u',
    'û' : 'u',
    'ǔ' : 'u',
    'ū' : 'u',
    'ű' : 'u',
    'u̍h' : 'uh',
    'u̍' : 'u',


    'ḿ' : 'm',
    'm̀' : 'm',
    'm̂' : 'm',
    'm̌' : 'm',
    'm̄' : 'm',
    'm̋' : 'm',
    'm̍h' : 'mh',
    'm̍' : 'm',


    'ń' : 'n',
    'ǹ' : 'n',
    'n̂' : 'n',
    'ň' : 'n',
    'n̄' : 'n',
    'n̋' : 'n',
    'n̍h' : 'nh',
    'n̍' : 'n',

}


for i in range(len(groundtruth)):
    
    normalize_gt = groundtruth[i]
    normalize_hp = hypothesis[i]

    for k, v in replace_map.items():
        normalize_gt = normalize_gt.replace(k, v)
        normalize_hp = normalize_hp.replace(k, v)

    normal_groundtruth.append(normalize_gt)
    normal_hypothesis.append(normalize_hp)



In [28]:
print(len(normal_groundtruth), len(normal_hypothesis))

wer = wer_cal(normal_groundtruth, normal_hypothesis)
cer = cer_cal(normal_groundtruth, normal_hypothesis)
print()
print(wer)
print(cer)

5837 5837

0.1411325647405123
0.06072567378639064


In [29]:
for i in range(10):
    print(normal_groundtruth[i])
    print(normal_hypothesis[i])
    print(wer_cal([normal_groundtruth[i]], [normal_hypothesis[i]]))
    input()


gua e hoo tsio ho be si kiu tshit pat ji pat ji khong kiu khong
gua e hoo tsioh ho be si kiu tshit pat ji pat ji khong kiu khong
0.0625
tsit puann si siann mih tshai
tsit puann si siann mih tshai
0.0
a tsing a li tsit e gin a si an tsuann
a tsing a li tsit e gin a si an tsuann
0.0
siok tsu kin me tshih tiau
siok tsu kim me tshi tiau
0.3333333333333333
than bo tsiah
than bo tsiah
0.0
kam u lang tsai iann bun tsing tshu li e tsing hing
kam m u lang tsai iann bun tsing tshu lai e tsing hing
0.16666666666666666
tioh ai ku ku ah tsiah u tsit nia sin sann thang tshing
tioh ai ku a tsiah u tsit iann sin sann thang tsing
0.3076923076923077
tan bo sann siann
tam bo sann sia
0.5
tiam tiam leh ka lau hu jin lang tau tsing li tsua phe
tiam leh ka lau hu lin lang tau tsing li tsua phe
0.15384615384615385


KeyboardInterrupt: Interrupted by user